# Stochastic simulation of Arp2/3 complex recruitment to NPF-coated surface



In [1]:
from numpy import zeros, sqrt, ceil, int, exp
from numpy.random import rand

In [2]:
npf_density = 1000 # in molecules per square micron
square_length = 1.0 # in microns
'''
k_on_solution = 16E7 /M/s is a rate estimated from fluorescence anisotropy measurements in solution.
If the rate is limited by diffusion, then perhaps I can adjust the constant:
'''
k_on = 16 * 10**7 * (220 / (220 + 2 * 42 + 2 * 8))**(1/3) * 50 * 10**-9
# k_act = 5.0**-1 # in states per second, may depend on acidity of NPF CA region
k_off = 0.7**-1 # in molecules per second, may be accelerated by a protrusive network

In [3]:
no_npfs_side = int(sqrt(npf_density))
npf_state_mat = zeros((no_npfs_side, no_npfs_side), dtype = int)
'''
0: unbound state
1: inactive state
2: active state
'''

time_interval = 1e-3 # in seconds
simulation_time = 3.0 # in seconds

def simulate_surface_nucleation(k_act):
    current_time = 0.0 # in seconds
    no_ends = 0

    while current_time <= simulation_time:
        # Iterate. Maybe vectorize at some point?
        for i_npf in range(no_npfs_side):
            for j_npf in range(no_npfs_side):
                if npf_state_mat[i_npf, j_npf] == 0:
                    if exp(-k_on * time_interval) < rand():
                        npf_state_mat[i_npf, j_npf] = 1
                elif npf_state_mat[i_npf, j_npf] == 1:
                    if exp(-k_off * time_interval) < rand():
                        npf_state_mat[i_npf, j_npf] = 0
                        continue
                    elif exp(-k_act * time_interval) < rand():
                        npf_state_mat[i_npf, j_npf] = 2
                elif npf_state_mat[i_npf, j_npf] == 2:
                    if exp(-k_off * time_interval) < rand():
                        npf_state_mat[i_npf, j_npf] = 0
                        no_ends += 1
                        continue
        current_time += time_interval
    nucleation_rate = no_ends / current_time
    return nucleation_rate

In [4]:
'''
from matplotlib.pyplot import figure, subplots
from skimage.color import label2rgb
figure_handle, axes_handle = subplots()
axes_handle.imshow(label2rgb(npf_state_mat, colors = [[1, 1, 1], [1, 0, 0], [0, 0, 1]]))
axes_handle.axis('off')
'''

"\nfrom matplotlib.pyplot import figure, subplots\nfrom skimage.color import label2rgb\nfigure_handle, axes_handle = subplots()\naxes_handle.imshow(label2rgb(npf_state_mat, colors = [[1, 1, 1], [1, 0, 0], [0, 0, 1]]))\naxes_handle.axis('off')\n"

In [ ]:
from numpy import linspace
k_act_row = linspace(0.5, 10.0, 20)**-1
nucleation_rate_row = zeros(20)
for i in range(20):
    nucleation_rate_row[i] = simulate_surface_nucleation(k_act_row[i])

In [ ]:
from matplotlib.pyplot import figure, subplots
fig1_handle, axes1_handle = subplots()
axes1_handle.plot(k_act_row, nucleation_rate_row)
axes1_handle.set_xlabel("activation rate (/s)", fontsize = 12)
axes1_handle.set_ylabel("Nucleation rate (/s)", fontsize = 12)